In [3]:
###5.5.1. 加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [4]:
x2 = torch.load('x-file')
x2

C:\Users\yy200\AppData\Local\Temp\ipykernel_6544\1444019752.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2 = torch.load('x-file')


tensor([0, 1, 2, 3])

In [5]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

C:\Users\yy200\AppData\Local\Temp\ipykernel_6544\2924237495.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x2, y2 = torch.load('x-files')


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [6]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

C:\Users\yy200\AppData\Local\Temp\ipykernel_6544\2269971588.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mydict2 = torch.load('mydict')


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [7]:
###5.5.2. 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)    


In [8]:
torch.save(net.state_dict(), 'mlp.params')

In [9]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

C:\Users\yy200\AppData\Local\Temp\ipykernel_6544\2014828273.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clone.load_state_dict(torch.load('mlp.params'))


MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [10]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [17]:
#假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
# 加载预训练模型
original_net = MLP() 
original_net.load_state_dict(torch.load('mlp.params'))

class NewNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 复用前两层（包含ReLU激活）
        self.pretrained = nn.Sequential(
            original_net.hidden,  # 原始hidden层: 20 -> 256
            nn.ReLU()             # 添加原网络中的激活函数
        )
        
        # 冻结复用层参数（修正冻结方式）
        for param in self.pretrained.parameters():
            param.requires_grad = False
            
        # 修正维度匹配（根据hidden层输出256维）
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),  # 输入维度调整为256
            nn.ReLU(),
            nn.Linear(128, 10))
            
    def forward(self, x):
        x = self.pretrained(x)
        return self.classifier(x)

new_net = NewNet()
X = torch.randn(size=(1, 20))
Y = new_net(X)
torch.save(new_net.state_dict(), 'new_mlp.params')

C:\Users\yy200\AppData\Local\Temp\ipykernel_6544\3266544823.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  original_net.load_state_dict(torch.load('mlp.params'))


In [20]:
#如何同时保存网络架构和参数？需要对架构加上什么限制？
# 保存完整模型（包含架构+参数）
model = MLP()
torch.save({
    'arch': MLP,              # 保存类定义
    'state_dict': model.state_dict(),  # 保存初始化参数
}, 'full_model.pth')

# 加载时需要满足的条件
def load_model(path):
    checkpoint = torch.load(path)
    # 必须能访问原始类定义
    model = checkpoint['arch'](**checkpoint['metadata']) 
    model.load_state_dict(checkpoint['state_dict'])
    return model